In [23]:
from dotenv import load_dotenv

load_dotenv()

True

In [33]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [25]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

# 데이터 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

# 파서 설정
output_parser = JsonOutputParser(pydantic_object=Topic)

In [26]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [20]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [31]:

from langchain_core.prompts import ChatPromptTemplate

question = "점심먹고 졸린 이유를 알려주세요."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format: {format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

In [34]:
chain = prompt | model | output_parser

answer = chain.invoke({"question": question})

In [35]:
type(answer)

dict

In [36]:
print(answer)

{'description': '점심 식사 후 졸림은 주로 혈당 수치의 변화와 소화 과정으로 인한 에너지 소모 때문입니다. 식사 후 혈당이 급격히 상승하면 인슐린이 분비되어 혈당을 낮추고, 이 과정에서 세로토닌과 멜라토닌 같은 호르몬이 증가하여 졸음을 유발할 수 있습니다. 또한, 소화 과정에서 많은 에너지가 사용되면서 몸이 피로를 느낄 수 있습니다.', 'hashtags': '#점심졸림 #혈당변화'}


In [37]:
output_parser = JsonOutputParser()

question = "잠에서 깨는 방법에 대해 알려주세요. 방법에 대한 설명은 `descrikption`에 관련키워드는 `hashtags`에 담아주세요"

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format: {format_instructions}\nQuestion:\n{question}")
    ]
)

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | model | output_parser

response = chain.invoke({"question" : question})

print(response)


{'methods': [{'description': '자연광을 이용해 아침에 일어나기', 'hashtags': ['#자연광', '#아침', '#일어나기']}, {'description': '알람 소리를 점차 크게 설정하기', 'hashtags': ['#알람', '#소리', '#점차크게']}, {'description': '아침 스트레칭으로 몸을 깨우기', 'hashtags': ['#스트레칭', '#아침', '#몸깨우기']}, {'description': '커피나 차와 같은 음료로 정신을 깨우기', 'hashtags': ['#커피', '#차', '#정신깨우기']}, {'description': '찬물로 얼굴을 씻어 상쾌함 느끼기', 'hashtags': ['#찬물', '#얼굴씻기', '#상쾌함']}]}
